# Face Recognition Model

In [8]:
# Cell 1: Import necessary libraries
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='keras')
import os
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

print("All imports loaded successfully.")

# Cell 2: Set paths to train and test directories
train_directory = os.path.join('C:/Users/reeva/Desktop/690/fer-2013', 'train')
test_directory = os.path.join('C:/Users/reeva/Desktop/690/fer-2013', 'test')
model_path = 'best_model.keras'  # Path to save/load the trained model

print("Paths set: \nTrain Directory:", train_directory, "\nTest Directory:", test_directory)

# Cell 3: Check if the model exists
if os.path.exists(model_path):
    print(f"Loading saved model from {model_path}...")
    model = load_model(model_path)
    print("Model loaded successfully.")
else:
    # Cell 4: Create ImageDataGenerators for training and testing with more augmentation
    try:
        train_datagen = ImageDataGenerator(
            rescale=1.0/255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            brightness_range=[0.8, 1.2]
        )

        test_datagen = ImageDataGenerator(rescale=1.0/255)
        print("ImageDataGenerators created successfully.")
    except Exception as e:
        print("Error in creating ImageDataGenerators:", e)

    # Cell 5: Set batch size and create train/test generators
    batch_size = 32

    try:
        # Use RGB to match VGG16 input expectations
        train_generator = train_datagen.flow_from_directory(
            train_directory,
            target_size=(48, 48),
            color_mode='rgb',
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True
        )
        print("Training generator created successfully.")
        print(f"Number of training samples: {train_generator.samples}")
        print(f"Number of classes (labels): {train_generator.num_classes}")
        print(f"Class labels: {train_generator.class_indices}")
    except Exception as e:
        print("Error in creating training generator:", e)

    try:
        test_generator = test_datagen.flow_from_directory(
            test_directory,
            target_size=(48, 48),
            color_mode='rgb',
            batch_size=batch_size,
            class_mode='categorical'
        )
        print("Test generator created successfully.")
        print(f"Number of test samples: {test_generator.samples}")
        print(f"Number of classes (labels): {test_generator.num_classes}")
    except Exception as e:
        print("Error in creating test generator:", e)

    # Cell 6: Fine-tune some VGG16 layers and define the new model
    try:
        print("Loading VGG16 base model...")
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

        # Unfreeze the last few convolutional layers to allow fine-tuning
        for layer in base_model.layers[-4:]:
            layer.trainable = True

        model = Sequential()
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(7, activation='softmax'))

        print("VGG16 base model loaded and new model defined successfully.")
        
        # Compile the model with Adam optimizer
        print("Compiling the model...")
        model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
        print("Model compiled successfully.")
    except Exception as e:
        print("Error in defining/compiling the model:", e)

    # Cell 7: Set up callbacks for saving the best model, early stopping, and reducing the learning rate on plateau
    checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

    # Cell 8: Fit the model without class weights
    try:
        print("Starting model training without class weights...")
        history = model.fit(
            train_generator,
            validation_data=test_generator,
            epochs=50,
            steps_per_epoch=train_generator.samples // batch_size,
            validation_steps=test_generator.samples // batch_size,
            callbacks=[checkpoint, early_stopping, reduce_lr],
            verbose=1  # Set verbosity to see training progress
        )
        best_accuracy = max(history.history['val_accuracy'])
        print(f"Model training completed with highest validation accuracy: {best_accuracy * 100:.2f}%")
    except Exception as e:
        print("Error during model training:", e)

    # The model is saved using ModelCheckpoint callback
    print(f"Training complete. Model saved as '{model_path}'.")

# Cell 9: Load the face classifier (for real-time inference later)
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
print("Face classifier loaded successfully.")

# Cell 10: Define a decorated function for prediction to avoid retracing
@tf.function
def predict_emotion(roi):
    return model(roi, training=False)

# Cell 11: Define function for detecting and predicting emotions
def detect_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            # Convert grayscale image to RGB by stacking the same channel 3 times
            roi_rgb = np.stack((roi_gray,) * 3, axis=-1)
            roi = roi_rgb.astype('float32') / 255.0
            roi = np.expand_dims(roi, axis=0)  # Add batch dimension

            prediction = predict_emotion(roi)[0].numpy()  # Use decorated function
            emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
            emotion_results = {emotions[idx]: prediction[idx] * 100 for idx in range(len(emotions))}
            
            # Print emotions and their confidence percentages
            for emotion, confidence in emotion_results.items():
                print(f"{emotion}: {confidence:.2f}%")
                
            return emotion_results

    return None

# Cell 12: Capture image from webcam and detect emotion
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
else:
    print("Webcam accessed successfully. Press 'c' to capture a photo.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow('Emotion Detector - Press "c" to capture', frame)

    # Press 'c' to capture the photo and analyze it
    if cv2.waitKey(1) & 0xFF == ord('c'):
        print("Photo captured. Analyzing...")
        emotion_results = detect_emotion(frame)
        if emotion_results:
            print("Emotion detection completed successfully.")
        else:
            print("No face detected. Please try again.")
        break

cap.release()
cv2.destroyAllWindows()
print("Webcam feed closed.")


All imports loaded successfully.
Paths set: 
Train Directory: C:/Users/reeva/Desktop/690/fer-2013\train 
Test Directory: C:/Users/reeva/Desktop/690/fer-2013\test
Loading saved model from best_model.keras...
Model loaded successfully.
Face classifier loaded successfully.
Webcam accessed successfully. Press 'c' to capture a photo.
Photo captured. Analyzing...
Angry: 91.76%
Disgust: 1.88%
Fear: 5.89%
Happy: 0.06%
Sad: 0.02%
Surprise: 0.26%
Neutral: 0.13%
Emotion detection completed successfully.
Webcam feed closed.
